In [1]:
import numpy as np
import pandas as pd

path='../../foothill/raw/'
date_list=['2021-04-19','2021-04-20','2021-04-21','2021-05-05','2021-05-06']

meta=pd.read_csv('../meta/Hwy20_detectors.csv')



In [48]:
occupancy_of_chunk_150_lenght[(occupancy_of_chunk_150_lenght['signal']=='JFK') ]['detector_parameter'].unique()

array([33., 34., 35., 36., 37., 39., 41., 43., 45., 47., 49., 50., 51.,
       52., 53., 54., 55., 56., 57., 58., 59., 60., 61., 62., 63.])

In [3]:
meta=meta[['Intersection','Phase','Detector','Detection Type','Detection Hardware']].drop_duplicates()
meta['Detection Type']=np.where(meta['Detection Type']=='Normal','Stop Bar Presence','Lane-by-lane Count')
meta[meta['Intersection']=='Century']['Detection Type'].unique()

array(['Stop Bar Presence', 'Lane-by-lane Count'], dtype=object)

In [4]:
manual=pd.DataFrame([])
for d in date_list:

    occupancy_of_chunk_150_lenght=pd.read_csv('../occupancy/'+d+'/'+'occupancy_of_chunk_150_lenght'+d+'.csv', index_col=[0])


    occupancy_of_chunk_150_lenght=occupancy_of_chunk_150_lenght.sort_values(['signal','detector_parameter','unique_cycle_id','chunk'])
    occupancy_of_chunk_150_lenght['below_second']=occupancy_of_chunk_150_lenght.groupby(['signal','detector_parameter','unique_cycle_id'])['seconds'].shift(-1)

    occupancy_of_chunk_150_lenght['point_difference']=occupancy_of_chunk_150_lenght['below_second']-occupancy_of_chunk_150_lenght['seconds']
    occupancy_of_chunk_150_lenght.start_cycle=pd.to_datetime(occupancy_of_chunk_150_lenght.start_cycle)
    occupancy_of_chunk_150_lenght['date']=occupancy_of_chunk_150_lenght.start_cycle.dt.date


    temp=occupancy_of_chunk_150_lenght[['signal','date','detector_parameter','unique_cycle_id','chunk','point_difference']]
    temp=temp[temp.point_difference<0]

    temp=temp.sort_values(['signal','date','detector_parameter','unique_cycle_id','chunk']).assign(
    first_ocup_drop_chunk = 
    temp
    .groupby(['signal','date','detector_parameter','unique_cycle_id'])
    .chunk
    .transform('min')
    )

    temp=temp.assign(
    max_occup_drop_value = 
    temp
    .groupby(['signal','date','detector_parameter','unique_cycle_id'])
    .point_difference
    .transform('min')
    )


    temp['drop_point']=np.where(temp.max_occup_drop_value==temp.point_difference, 'maximum_drop_point','na' )


    #alternative_solution_phase=pd.read_csv('../../phase_prob/'+d+'/'+'adjustment_left_merged_1'+d+'.csv', index_col=[0])
    alternative_solution_phase=pd.read_csv('../phase_prob/'+d+'/'+'alternative_solution_phase_probability'+d+'.csv', index_col=[0])

    alternative_solution_phase.cycle_start=pd.to_datetime(alternative_solution_phase.cycle_start)
    alternative_solution_phase['date']=alternative_solution_phase.cycle_start.dt.date


    temp2=alternative_solution_phase[(alternative_solution_phase['event_color']=='Green') & (alternative_solution_phase['location']=='first') ].sort_values(['signal','parameter','unique_cycle_id','b']).groupby(['signal','parameter','unique_cycle_id']).first().reset_index()
    temp2=temp2[['signal', 'date','parameter', 'cycle_id', 'unique_cycle_id', 'event_color',
       'location', 'b',  'below_chunk', 'event_start_time',
       'event_end_time', 'cycle_start', 'cycle_end', 'event', 'chunk_start',
       'chunk_end']]


    combination=temp2.merge(temp, how='left', on=['signal','unique_cycle_id','date'])[['signal','date','parameter','detector_parameter','cycle_id','unique_cycle_id','chunk','b','first_ocup_drop_chunk','max_occup_drop_value','drop_point']]
    combination['chunk_difference']=combination['chunk']-combination['b']
    combination=combination[( combination['chunk_difference']<=2) & ( combination['chunk_difference']>=0) ].rename(columns={'chunk':'occupancy_drop_chunk','b':'phase_first_green_chunk'})
    combination=combination.reset_index(drop=True)
    combination['drop_id']=combination.index

    combination=combination.merge(meta, left_on=['signal','detector_parameter'], right_on=['Intersection','Detector'], how='left')
    method1=combination.groupby(['signal','date','detector_parameter','Phase','parameter','chunk_difference'])['drop_id'].count().reset_index()
    method1=method1.rename(columns={'drop_id':'count_of_drop'})

    method2=method1.assign(
        max_count_drop = 
        method1
        .groupby(['signal','date','detector_parameter','Phase','chunk_difference'])
        .count_of_drop
        .transform('max')
        )

    method3=method2[method2['max_count_drop']==method2['count_of_drop']].sort_values(['signal','date','detector_parameter','Phase','chunk_difference','max_count_drop'])

    method4=method3.assign(
    frequent_parameter_count= 
    method3
    .groupby(['signal','date','detector_parameter','parameter'])
    .chunk_difference
    .transform('count')
    )



    #method4=method3.groupby(['signal','detector_parameter','parameter'])['chunk_difference'].count().reset_index()
    method5=method4.drop_duplicates(['signal','date','detector_parameter','Phase','parameter']).sort_values(['signal','detector_parameter','Phase','parameter'])

    method6=method5[['signal','date','detector_parameter','Phase','parameter','frequent_parameter_count']].assign(
    max_of_frequent_parameter_count= 
    method5
    .groupby(['signal','date','detector_parameter','Phase'])
    .frequent_parameter_count
    .transform('max')
    )

    method7=method6[method6['frequent_parameter_count']==method6['max_of_frequent_parameter_count']]
    manual=manual.append(method7)

In [5]:
manual['correct']='na'
manual['correct']=np.where(manual['Phase']==manual['parameter'],1,0)

In [6]:
manual.to_csv('manual.csv')

In [70]:
manual=pd.read_csv('manual.csv', index_col=[0])

In [35]:
summary=manual.groupby(['signal','detector_parameter','parameter','Phase'])['correct'].count().reset_index().rename(columns={'correct':'count'})
summary.sort_values(['signal','detector_parameter','parameter','count'])

,signal,detector_parameter,parameter,Phase,count
0,Century,33.0,2.0,2.0,2
1,Century,33.0,6.0,2.0,3
2,Century,35.0,2.0,5.0,2
3,Century,35.0,5.0,5.0,5
4,Century,37.0,4.0,4.0,5
...,...,...,...,...,...
291,Wacker,45.0,4.0,8.0,3
292,Wacker,45.0,5.0,8.0,2
293,Wacker,45.0,8.0,8.0,4
294,Wacker,47.0,3.0,3.0,5


In [36]:
summary.to_csv('summary.csv')

In [34]:
summary[summary['signal']=='JFK']['detector_parameter'].nunique()

24

In [8]:
summary=summary.merge(meta, left_on=['signal','detector_parameter','Phase'], right_on=['Intersection','Detector','Phase'])
summary['Detection Type'].unique()

array(['Stop Bar Presence', 'Lane-by-lane Count'], dtype=object)

In [25]:
summary[summary['signal']=='JFK']['detector_parameter'].nunique()

24

In [27]:
result=summary.loc[summary.groupby(['signal','detector_parameter'])['count'].idxmax()]
result['correct']=np.where(result['parameter']==result['Phase'],1,0)
result

,signal,detector_parameter,parameter,Phase,count,Intersection,Detector,Detection Type,Detection Hardware,correct
1,Century,33.0,6.0,2.0,3,Century,33.0,Stop Bar Presence,Matrix,0
3,Century,35.0,5.0,5.0,5,Century,35.0,Stop Bar Presence,Matrix,1
4,Century,37.0,4.0,4.0,5,Century,37.0,Stop Bar Presence,Matrix,1
6,Century,39.0,7.0,7.0,5,Century,39.0,Stop Bar Presence,Matrix,1
7,Century,41.0,1.0,6.0,5,Century,41.0,Stop Bar Presence,Matrix,0
...,...,...,...,...,...,...,...,...,...,...
285,Wacker,39.0,3.0,7.0,5,Wacker,39.0,Stop Bar Presence,Matrix,0
290,Wacker,41.0,6.0,6.0,4,Wacker,41.0,Stop Bar Presence,Matrix,1
291,Wacker,43.0,1.0,1.0,5,Wacker,43.0,Stop Bar Presence,Matrix,1
296,Wacker,45.0,8.0,8.0,4,Wacker,45.0,Stop Bar Presence,Matrix,1


In [31]:
result[result['signal']=='JFK']

,signal,detector_parameter,parameter,Phase,count,Intersection,Detector,Detection Type,Detection Hardware,correct
127,JFK,33.0,6.0,2.0,5,JFK,33.0,Stop Bar Presence,Matrix,0
128,JFK,35.0,2.0,5.0,5,JFK,35.0,Stop Bar Presence,Matrix,0
130,JFK,36.0,3.0,2.0,5,JFK,36.0,Stop Bar Presence,Advanced,0
131,JFK,37.0,3.0,3.0,4,JFK,37.0,Stop Bar Presence,Matrix,1
135,JFK,39.0,3.0,3.0,5,JFK,39.0,Stop Bar Presence,Matrix,1
136,JFK,41.0,4.0,4.0,5,JFK,41.0,Stop Bar Presence,Matrix,1
137,JFK,43.0,4.0,4.0,5,JFK,43.0,Stop Bar Presence,Matrix,1
138,JFK,45.0,1.0,6.0,5,JFK,45.0,Stop Bar Presence,Matrix,0
139,JFK,47.0,1.0,1.0,5,JFK,47.0,Stop Bar Presence,Matrix,1
140,JFK,49.0,2.0,5.0,5,JFK,49.0,Lane-by-lane Count,Matrix,0


In [10]:
result=result[result['Phase']<9]
#result=result[result['Detection Hardware']!='Advanced']

In [11]:
result=result[~result['Detection Hardware'].isin(['Advanced ','Advanced'])]
result['Detection Hardware'].unique()

array(['Matrix'], dtype=object)

In [12]:
result['Detection Type'].unique()

array(['Stop Bar Presence', 'Lane-by-lane Count'], dtype=object)

In [13]:
result.to_csv('result.csv')

In [16]:
result.groupby(['signal','Detection Type'])['detector_parameter'].count().reset_index()

,signal,Detection Type,detector_parameter
0,Century,Lane-by-lane Count,12
1,Century,Stop Bar Presence,8
2,Crescent Ridge,Lane-by-lane Count,14
3,Crescent Ridge,Stop Bar Presence,4
4,Devon,Lane-by-lane Count,13
5,Devon,Stop Bar Presence,8
6,JFK,Lane-by-lane Count,14
7,JFK,Stop Bar Presence,8
8,NW Arterial,Lane-by-lane Count,14
9,NW Arterial,Stop Bar Presence,8


In [18]:
result

,signal,detector_parameter,parameter,Phase,count,Intersection,Detector,Detection Type,Detection Hardware,correct
1,Century,33.0,6.0,2.0,3,Century,33.0,Stop Bar Presence,Matrix,0
3,Century,35.0,5.0,5.0,5,Century,35.0,Stop Bar Presence,Matrix,1
4,Century,37.0,4.0,4.0,5,Century,37.0,Stop Bar Presence,Matrix,1
6,Century,39.0,7.0,7.0,5,Century,39.0,Stop Bar Presence,Matrix,1
7,Century,41.0,1.0,6.0,5,Century,41.0,Stop Bar Presence,Matrix,0
...,...,...,...,...,...,...,...,...,...,...
285,Wacker,39.0,3.0,7.0,5,Wacker,39.0,Stop Bar Presence,Matrix,0
290,Wacker,41.0,6.0,6.0,4,Wacker,41.0,Stop Bar Presence,Matrix,1
291,Wacker,43.0,1.0,1.0,5,Wacker,43.0,Stop Bar Presence,Matrix,1
296,Wacker,45.0,8.0,8.0,4,Wacker,45.0,Stop Bar Presence,Matrix,1


In [14]:
g=result.groupby('signal')
detailed_accuracy=pd.DataFrame([])
for sets, data in g:
    accuracy=len(data[data['correct']==1])/len(data)
    df=pd.DataFrame({'signal':sets, 'accuracy':accuracy,'number of detectors':len(data)}, index=[0])
    detailed_accuracy=detailed_accuracy.append(df)
detailed_accuracy

,signal,accuracy,number of detectors
0,Century,0.700000,20
0,Crescent Ridge,0.666667,18
0,Devon,0.476190,21
0,JFK,0.500000,22
0,NW Arterial,0.681818,22
0,University,0.578947,19
0,Wacker,0.750000,8


In [19]:
g=result.groupby(['signal','Detection Type'])
detailed_accuracy=pd.DataFrame([])
for sets, data in g:
    accuracy=len(data[data['correct']==1])/len(data)
    df=pd.DataFrame({'signal':sets[0],'Detection Type':sets[1] ,'accuracy':accuracy,'number of detectors':len(data)}, index=[0])
    detailed_accuracy=detailed_accuracy.append(df)
detailed_accuracy

,signal,Detection Type,accuracy,number of detectors
0,Century,Lane-by-lane Count,0.666667,12
0,Century,Stop Bar Presence,0.750000,8
0,Crescent Ridge,Lane-by-lane Count,0.714286,14
0,Crescent Ridge,Stop Bar Presence,0.500000,4
0,Devon,Lane-by-lane Count,0.538462,13
0,Devon,Stop Bar Presence,0.375000,8
0,JFK,Lane-by-lane Count,0.428571,14
0,JFK,Stop Bar Presence,0.625000,8
0,NW Arterial,Lane-by-lane Count,0.714286,14
0,NW Arterial,Stop Bar Presence,0.625000,8


# after removing the low data and now signal channels

In [20]:
result_filtered=pd.read_csv('result_filtered.csv')

In [21]:
import pandas as pd

g=result_filtered.groupby('signal')
detailed_accuracy=pd.DataFrame([])
for sets, data in g:
    accuracy=len(data[data['correct']==1])/len(data)
    df=pd.DataFrame({'signal':sets, 'accuracy':accuracy,'number of detectors':len(data)}, index=[0])
    detailed_accuracy=detailed_accuracy.append(df)
detailed_accuracy
#century signal is with left_merged_   for signal color the following cell
#Devon signal as well

,signal,accuracy,number of detectors
0,Century,0.875000,16
0,Crescent Ridge,0.857143,14
0,Devon,0.588235,17
0,JFK,0.785714,14
0,NW Arterial,0.789474,19
0,University,0.733333,15
0,Wacker,0.750000,8
